In [1]:
'''装环境'''
# pip install pyqt5 matplotlib networkx scipy numpy


'装环境'

In [2]:
'''
读取点云和marks
对点云插值加密并中心化
合并不同文件的marks
'''

import chardet
import numpy as np
import networkx as nx
import scipy.interpolate as spi
import matplotlib.pyplot as plt
from scipy.spatial import KDTree

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        return result['encoding']
        
def load_obj_file(file_path, encoding):
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return vertices, faces

def load_mark_file(file_path, encoding):
    marks = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) == 3:
                    mark = [float(parts[0]), float(parts[1]), float(parts[2])]
                    marks.append(mark)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return marks

def insert_midpoint_points(vertices, faces, minimum_vertices_number=5000):
    if len(vertices) >= minimum_vertices_number:
        print('原始点云的点数足够多')
        return vertices, faces

    else:
        temp = 0
        while len(vertices) < minimum_vertices_number:
            edge_to_midpoint = {}
            new_points = []
            new_faces = []
            vertex_offset = len(vertices)
    
            for face in faces:
                # Compute midpoints for each edge
                midpoints = []
                for i in range(len(face)):
                    edge = tuple(sorted((face[i], face[(i + 1) % len(face)])))
                    if edge not in edge_to_midpoint:
                        midpoint = np.mean([vertices[edge[0]], vertices[edge[1]]], axis=0)
                        edge_to_midpoint[edge] = vertex_offset + len(new_points)
                        new_points.append(midpoint.tolist())
                    midpoints.append(edge_to_midpoint[edge])
                
                # Original vertices
                v0, v1, v2 = face
                # Midpoints
                m0, m1, m2 = midpoints

                # Create four new faces
                new_faces.append([v0, m0, m2])
                new_faces.append([v1, m1, m0])
                new_faces.append([v2, m2, m1])
                new_faces.append([m0, m1, m2])
    
            vertices.extend(new_points)
            faces = new_faces
            temp += 1

        print(f'使用了 {temp} 轮插值来让点云点数满足要求')
        
    return vertices, faces

def center_vertices(vertices):
    vertices_array = np.array(vertices)
    min_coords = vertices_array.min(axis=0)
    max_coords = vertices_array.max(axis=0)
    center = (min_coords + max_coords) / 2
    centered_vertices = vertices_array - center
    return centered_vertices.tolist()



In [3]:
'''对点云和marks执行姿态调整'''

import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import UnivariateSpline
import os
import matplotlib.pyplot as plt

   
def compute_bounding_box(points):
    """计算点云的包围盒"""
    min_point = np.min(points, axis=0)
    max_point = np.max(points, axis=0)
    bbox_size = max_point - min_point
    bbox_center = (min_point + max_point) / 2.0
    return bbox_size, bbox_center

def extract_top_subcloud(points, bbox_size, bbox_center, height=4.0):
    """提取y坐标在ymax到ymax-3范围内的子点云"""
    min_y = bbox_center[1] + bbox_size[1] / 2.0 - height
    top_subcloud = points[points[:, 1] >= min_y]
    return top_subcloud

def find_best_rotation(points, axis1, axis2, axis3, half_angle_range=int(90), angle_step=1):
    """绕axis1旋转，使得axis2（优先）、axis3方向包围盒尺寸最小，输出旋转矩阵"""
    min_size2 = float('inf')
    min_size3 = float('inf')
    best_rotation_matrix = None

    for angle in np.arange(-half_angle_range, half_angle_range, angle_step):
        # 绕指定轴旋转
        rotation_matrix = R.from_euler(axis1, angle, degrees=True).as_matrix()
        rotated_points = points.dot(rotation_matrix.T)
        
        # 计算包围盒尺寸
        bbox_size, _ = compute_bounding_box(rotated_points)
        
        # 找出目标轴之外的最小尺寸
        if axis2 == 'x':
            size2 = bbox_size[0]  # x方向宽度
        elif axis2 == 'y':
            size2 = bbox_size[1]  # z方向宽度
        elif axis2 == 'z':
            size2 = bbox_size[2]  # z方向宽度

        # 找出目标轴之外的最小尺寸
        if axis3 == 'x':
            size3 = bbox_size[0]  # x方向宽度
        elif axis3 == 'y':
            size3 = bbox_size[1]  # z方向宽度
        elif axis3 == 'z':
            size3 = bbox_size[2]  # z方向宽度

        # 如果当前尺寸比之前的最小尺寸还小，则更新
        if size2 < min_size2:
            min_size2 = size2
            min_size3 = size3
            best_rotation_matrix = rotation_matrix
        elif size2 == min_size2:
            if size3 <min_size3:
                min_size3 = size3
                best_rotation_matrix = rotation_matrix

    return best_rotation_matrix

def flatten_and_analyze_curve(points, z_threshold=0.3, y_offset=1.5):
    """在新x-y平面上分析曲线的曲率，并确定x、z方向。
    
    参数:
    - points: 点云数据，形状为 (n, 3)
    - z_threshold: 用于滤波的z坐标阈值
    - y_offset: 确定用于分裂曲线的横线 y=ymax-y_offset 的偏移量
    """
    # 提取平面内的点
    flat_points = points[np.abs(points[:, 2]) <= z_threshold]

    # 提取 x 和 y 坐标
    x_coords = flat_points[:, 0]
    y_coords = flat_points[:, 1]

    # 对 x_coords 和 y_coords 进行排序以确保 x_coords 严格递增
    sorted_indices = np.argsort(x_coords)
    x_coords = x_coords[sorted_indices]
    y_coords = y_coords[sorted_indices]

    # 确定 ymax - y_offset 这条横线
    ymax = np.max(y_coords)
    y_threshold = ymax - y_offset

    # 找到 y_coords 刚刚超过 y_threshold 的最小和最大索引
    above_threshold_indices = np.where(y_coords > y_threshold)[0]
    if len(above_threshold_indices) == 0:
        raise ValueError("没有点的y坐标超过指定的y_threshold")

    min_index = above_threshold_indices[0]
    max_index = above_threshold_indices[-1]

    # 使用这些索引作为左侧和右侧的分界点
    left_indices = np.where((y_coords <= y_threshold) & (x_coords <= x_coords[min_index]))[0]
    right_indices = np.where((y_coords <= y_threshold) & (x_coords >= x_coords[max_index]))[0]

    left_index = left_indices[np.argmin(np.abs(y_coords[left_indices] - y_threshold))]
    right_index = right_indices[np.argmin(np.abs(y_coords[right_indices] - y_threshold))]

    x2_left, y2_left = x_coords[left_index], y_coords[left_index]
    x2_right, y2_right = x_coords[right_index], y_coords[right_index]

    # 找到曲线的第一个点和最后一个点
    x1, y1 = x_coords[0], y_coords[0]
    x3, y3 = x_coords[-1], y_coords[-1]

    # 计算线段 (x1, y1)-(x2_left, y2_left) 和 (x2_right, y2_right)-(x3, y3) 的斜率
    slope_left = (y2_left - y1) / (x2_left - x1)
    slope_right = (y3 - y2_right) / (x3 - x2_right)

    # 计算两侧线段下方的点数占比
    left_side_points = flat_points[flat_points[:, 0] < x2_left]
    right_side_points = flat_points[flat_points[:, 0] >= x2_right]

    left_under_line = left_side_points[:, 1] < (slope_left * (left_side_points[:, 0] - x1) + y1)
    right_under_line = right_side_points[:, 1] < (slope_right * (right_side_points[:, 0] - x2_right) + y2_right)

    left_under_ratio = np.sum(left_under_line) / len(left_side_points)
    right_under_ratio = np.sum(right_under_line) / len(right_side_points)

    # 确定 x 方向：下凹一侧为 x 正方向
    x_direction = -1 if left_under_ratio > right_under_ratio else 1

    # 复制相同的旋转给 z 方向以保持手性
    z_direction = x_direction

    return x_direction, z_direction

def align_point_cloud(points):
    """处理并旋转点云"""  
    # 计算原始点云的包围盒
    bbox_size, bbox_center = compute_bounding_box(points)

    # 提取顶端子点云1：较短，用于找到最扁取向
    top_subcloud1 = extract_top_subcloud(points, bbox_size, bbox_center, height=3.0)

    # 找到使得点云最扁的旋转角度
    best_rotation_matrix = find_best_rotation(top_subcloud1, 'y', 'x', 'z')

    # 提取顶端子点云2：较长，用于对比唇舌侧的凸性
    top_subcloud2 = extract_top_subcloud(points, bbox_size, bbox_center, height=5.0)
    
    # 将子点云旋转到最佳角度
    rotated_subcloud = top_subcloud2.dot(best_rotation_matrix.T)
    
    # 在新x-y平面上分析曲线并确定x轴方向
    x_direction, z_direction = flatten_and_analyze_curve(rotated_subcloud)

    # 确定最终旋转矩阵
    new_x_axis = best_rotation_matrix[:, 0] * x_direction
    new_y_axis = best_rotation_matrix[:, 1]
    new_z_axis = best_rotation_matrix[:, 2] * z_direction

    best_rotation_matrix_y = np.column_stack((new_x_axis, new_y_axis, new_z_axis))

    # 旋转整个原始点云到新的坐标系
    aligned_points = points.dot(best_rotation_matrix_y.T)

    # 在z轴上进行进一步优化旋转
    best_rotation_matrix_z = find_best_rotation(aligned_points, 'z', 'x', 'y', half_angle_range=int(30))
    aligned_points = aligned_points.dot(best_rotation_matrix_z.T)

    # 在x轴上进行进一步优化旋转
    best_rotation_matrix_x = find_best_rotation(aligned_points, 'x', 'z', 'y', half_angle_range=int(30))
    aligned_points = aligned_points.dot(best_rotation_matrix_x.T)

    # 最终的旋转矩阵
    final_rotation_matrix = best_rotation_matrix_y.T.dot(best_rotation_matrix_z.T).dot(best_rotation_matrix_x.T)

    return aligned_points, final_rotation_matrix



In [4]:
import sys
import numpy as np
import networkx as nx
import csv
from scipy.spatial import KDTree
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton, QLabel, QHBoxLayout, QLineEdit, QFileDialog, QSpacerItem, QSizePolicy
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
from mpl_toolkits.mplot3d import Axes3D


def generate_topology(marks2):
    G = nx.Graph()
    tree = KDTree(marks2)

    for i, mark in enumerate(marks2):
        distances, indices = tree.query(mark.reshape(1, -1), k=3)
        for j in indices[0][1:]:
            G.add_edge(i, j)

    return G

class TopologyEditor(QMainWindow):
    def __init__(self, marks2, marks2_file_name):
        super().__init__()
        self.file_name = marks2_file_name
        
        self.marks2 = np.array(marks2)
        tree = KDTree(self.marks2)
        self.G = generate_topology(self.marks2)
        self.high_degree_list = np.array([node for node in self.G.nodes() if self.G.degree(node) >= 3])
        self.high_degree_list = self.high_degree_list[np.argsort(self.high_degree_list)]
        self.counter_High_Degree = 0  # 节点中心化计数
        
        self.node_colors = ['r'] * len(self.marks2)  # Default color for nodes is red
        self.edge_colors = {edge: 'k' for edge in self.G.edges()}

        self.node1 = 0
        self.node_colors[self.node1] = 'b'
        self.node2 = None

        self.zoom_factor = 1.0
        self.zoom_step = 0.2
        self.indices_size = 15  # 设置indices的字体大小
        self.indices_distance = 0.1 * np.array([0,1,0]) # 设置indices到对应点的距离

        self.setFocusPolicy(Qt.StrongFocus)  # 允许接收键盘事件
        self.counter_Left_and_Right = None  # 方向左右键计数

        # Initial view parameters
        self.azim = -60
        self.elev = 30
        
        self.initUI()
    
    def initUI(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        main_layout = QVBoxLayout(central_widget)
    
        self.canvas = FigureCanvas(Figure(figsize=(10, 8)))
        # Set size policy to expanding
        self.canvas.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Expanding)

        main_layout.addWidget(self.canvas)
        self.ax = self.canvas.figure.add_subplot(111, projection='3d')
        self.ax.view_init(elev=self.elev, azim=self.azim)
        
        # Create a widget for the input fields and buttons
        control_widget = QWidget()
        control_layout = QVBoxLayout(control_widget)
        control_layout.setAlignment(Qt.AlignCenter)
        control_layout.setSpacing(0)  # 设置控件之间的行距为最小
        
        # Create layouts for each row
        row0_layout = QHBoxLayout()
        row1_layout = QHBoxLayout()
        row2_layout = QHBoxLayout()
        row3_layout = QHBoxLayout()
        
        # 创建一个固定宽度的外部容器
        row0_container = QWidget()
        row0_container.setFixedWidth(1000)   # 设定行的固定宽度
        row0_container.setLayout(row0_layout)
        row1_container = QWidget()
        row1_container.setFixedWidth(1000)
        row1_container.setLayout(row1_layout)
        row2_container = QWidget()
        row2_container.setFixedWidth(1000)
        row2_container.setLayout(row2_layout)
        row3_container = QWidget()
        row3_container.setFixedWidth(1000)
        row3_container.setLayout(row3_layout)
        
        left_spacer = QSpacerItem(7, 0, QSizePolicy.Minimum, QSizePolicy.Minimum)  # 创建占位符
        self.status_label = QLabel('Enter two node indices to add or delete an edge.')
        middle_spacer1 = QSpacerItem(20, 0, QSizePolicy.Expanding, QSizePolicy.Minimum)
        self.high_degree_count = QLabel('High degree nodes counting.')
        middle_spacer2 = QSpacerItem(12, 0, QSizePolicy.Minimum, QSizePolicy.Minimum)
        self.center_next_button = QPushButton('Center on Next HDN')
        right_spacer = QSpacerItem(7, 0, QSizePolicy.Minimum, QSizePolicy.Minimum)
        self.center_next_button.setFixedWidth(200)  # Set fixed width for consistency
        
        self.node1_input = QLineEdit()
        self.node1_input.setPlaceholderText('Node 1')
        self.node1_confirm_button = QPushButton('Confirm Node 1')
        self.add_edge_button = QPushButton('Add Edge')
        self.center_node_button = QPushButton('Center on Node 1')
        self.node2_input = QLineEdit()
        self.node2_input.setPlaceholderText('Node 2')
        self.node2_confirm_button = QPushButton('Confirm Node 2')
        self.delete_edge_button = QPushButton('Delete Edge')
        self.reset_view_button = QPushButton('Reset View')
        self.node1_input.setFixedWidth(470)  # Set fixed width for consistency
        self.node1_confirm_button.setFixedWidth(150)  # Set fixed width for consistency
        self.add_edge_button.setFixedWidth(150)  # Set fixed width for consistency
        self.center_node_button.setFixedWidth(150)  # Set fixed width for consistency
        self.node2_input.setFixedWidth(470)  # Set fixed width for consistency
        self.node2_confirm_button.setFixedWidth(150)  # Set fixed width for consistency
        self.delete_edge_button.setFixedWidth(150)  # Set fixed width for consistency
        self.reset_view_button.setFixedWidth(150)  # Set fixed width for consistency      

        # Create a widget for zoom and view controls
        zoom_and_view_widget = QWidget()
    
        self.zoom_in_button = QPushButton('Zoom In')
        self.zoom_out_button = QPushButton('Zoom Out')
        self.reset_zoom_button = QPushButton('Reset Zoom')
        self.save_button = QPushButton('Save Data')
        self.load_button = QPushButton('Load Data')
        self.close_button = QPushButton('Close') 
        self.zoom_in_button.setFixedWidth(180)  # Set fixed width for consistency
        self.zoom_out_button.setFixedWidth(180)  # Set fixed width for consistency
        self.reset_zoom_button.setFixedWidth(180)  # Set fixed width for consistency
        self.save_button.setFixedWidth(180)  # Set fixed width for consistency
        self.load_button.setFixedWidth(180)  # Set fixed width for consistency
        self.close_button.setFixedWidth(150)  # Set fixed width for consistency
        
        row0_layout.addSpacerItem(left_spacer)   # 用占位符将status label向右推，实现对齐
        row0_layout.addWidget(self.status_label)
        row0_layout.addSpacerItem(middle_spacer1) # 隔开
        row0_layout.addWidget(self.high_degree_count)
        row0_layout.addSpacerItem(middle_spacer2) # 隔开
        row0_layout.addWidget(self.center_next_button)
        row0_layout.addSpacerItem(right_spacer)  # 向左推
        row1_layout.addWidget(self.node1_input)
        row1_layout.addWidget(self.node1_confirm_button)
        row1_layout.addWidget(self.add_edge_button)
        row1_layout.addWidget(self.center_node_button)
        row2_layout.addWidget(self.node2_input)
        row2_layout.addWidget(self.node2_confirm_button)
        row2_layout.addWidget(self.delete_edge_button)
        row2_layout.addWidget(self.reset_view_button)
        row3_layout.addWidget(self.save_button)
        row3_layout.addWidget(self.load_button)
        # row3_layout.addWidget(self.close_button)
        row3_layout.addWidget(self.zoom_in_button)
        row3_layout.addWidget(self.zoom_out_button)
        row3_layout.addWidget(self.reset_zoom_button)

        # 将4行的容器添加到主布局中
        control_layout.addWidget(row0_container)        
        control_layout.addWidget(row1_container)
        control_layout.addWidget(row2_container)
        control_layout.addWidget(row3_container)

        main_layout.addWidget(control_widget)
        main_layout.addWidget(zoom_and_view_widget)
    
        # Store the initial view limits
        self.initial_xlim = (self.marks2[:, 0].min() - 1, self.marks2[:, 0].max() + 1)
        self.initial_ylim = (self.marks2[:, 1].min() - 1, self.marks2[:, 1].max() + 1)
        self.initial_zlim = (self.marks2[:, 2].min() - 1, self.marks2[:, 2].max() + 1)
    
        # Initialize current view limits and center point
        self.current_xlim = list(self.initial_xlim)
        self.current_ylim = list(self.initial_ylim)
        self.current_zlim = list(self.initial_zlim)
        self.current_center = [
            (self.initial_xlim[0] + self.initial_xlim[1]) / 2,
            (self.initial_ylim[0] + self.initial_ylim[1]) / 2,
            (self.initial_zlim[0] + self.initial_zlim[1]) / 2
        ]
    
        self.plot_graph()

        # Connect buttons to their functions
        self.node1_confirm_button.clicked.connect(lambda: self.confirm_node(1))
        self.node2_confirm_button.clicked.connect(lambda: self.confirm_node(2))
        self.add_edge_button.clicked.connect(self.add_edge)
        self.delete_edge_button.clicked.connect(self.delete_edge)
        self.center_node_button.clicked.connect(self.center_on_node1)
        self.center_next_button.clicked.connect(self.move_center_to_next_node)
        self.reset_view_button.clicked.connect(self.reset_view)
        self.zoom_in_button.clicked.connect(self.zoom_in)
        self.zoom_out_button.clicked.connect(self.zoom_out)
        self.reset_zoom_button.clicked.connect(self.reset_zoom)
        self.save_button.clicked.connect(self.save_data)
        self.load_button.clicked.connect(self.load_data)
        # self.close_button.clicked.connect(self.close)

    def plot_graph(self):
        self.ax.clear()
        self.ax.set_xlabel('X Axis')
        self.ax.set_ylabel('Y Axis')
        self.ax.set_zlabel('Z Axis')
        
        # 计算当前视图范围
        x_min, x_max = self.current_xlim
        y_min, y_max = self.current_ylim
        z_min, z_max = self.current_zlim
    
        # 过滤在视图范围内的点
        mask = (
            (self.marks2[:, 0] >= x_min) & (self.marks2[:, 0] <= x_max) &
            (self.marks2[:, 1] >= y_min) & (self.marks2[:, 1] <= y_max) &
            (self.marks2[:, 2] >= z_min) & (self.marks2[:, 2] <= z_max)
        )
        filtered_indices = np.where(mask)[0]  # Extract the indices from the tuple
        
        # 绘制边
        for edge, color in self.edge_colors.items():
            if edge[0] in filtered_indices and edge[1] in filtered_indices:
                x = [self.marks2[edge[0], 0], self.marks2[edge[1], 0]]
                y = [self.marks2[edge[0], 1], self.marks2[edge[1], 1]]
                z = [self.marks2[edge[0], 2], self.marks2[edge[1], 2]]
                self.ax.plot(x, y, z, color=color)

        # Step 1: Calculate neighborhood radius
        neighborhood_radius = 2/ self.zoom_factor 
    
        # Step 2: Compute point cloud density
        tree = KDTree(self.marks2)
        densities = np.zeros(len(self.marks2))
    
        for i, mark in enumerate(self.marks2):
            distances, _ = tree.query(mark.reshape(1, -1), k=len(self.marks2))
            densities[i] = np.sum(distances[0] <= neighborhood_radius) - 1  # Exclude the point itself
    
        # Normalize densities to [0, 1]
        for i in range(len(densities)):
            densities[i] = min(1, densities[i] /5)

        # 定义节点
        self.high_degree_nodes = [node for node in self.G.nodes() if self.G.degree(node) >= 3]
        self.high_degree_count.setText(f'High degree nodes counting: {len(self.high_degree_nodes)}')
        
        # Step 3: Determine display probability and plot nodes
        for i in filtered_indices:
            self.node_colors[i] = 'limegreen' if i in self.high_degree_nodes and self.node_colors[i] == 'r' else self.node_colors[i]
            x, y, z = self.marks2[i]
            self.ax.scatter(x, y, z, c=self.node_colors[i], s=50)
    
            # Determine if label should be displayed based on density
            display_probability = self.zoom_factor - densities[i]  # Higher density -> lower probability, Bigger zoom factor -> Higher probability
            if np.random.rand() < display_probability:
                self.ax.text(x + self.indices_distance[0], y + self.indices_distance[1], z + self.indices_distance[2], str(i), color='k', fontsize=self.indices_size)

        # Additional functionality: Display nearest neighbors of node1
        if self.node1 is not None:
            # Find the nearest 10 neighbors of node1
            distances, indices = tree.query(self.marks2[self.node1].reshape(1, -1), k=11)
            nearest_indices = indices[0][0:]  # 含自身

            # Plot nearest neighbors
            for idx in nearest_indices:
                if idx in filtered_indices:
                    color = 'b' if idx == nearest_indices[0] else 'k'
                    mark = self.marks2[idx]
                    self.ax.text(mark[0] + self.indices_distance[0], mark[1] + self.indices_distance[1], mark[2] + self.indices_distance[2], str(idx), color=color, fontsize=self.indices_size)
        
        # 显示node2的index
        if self.node2 is not None:
            mark = self.marks2[self.node2]
            self.ax.text(mark[0] + self.indices_distance[0], mark[1] + self.indices_distance[1], mark[2] + self.indices_distance[2], str(self.node2), color='b', fontsize=self.indices_size)

        # 节点统一显示indices
        for idx in self.high_degree_nodes:
            if idx in filtered_indices:
                mark = self.marks2[idx]
                self.ax.text(mark[0] + self.indices_distance[0], mark[1] + self.indices_distance[1], mark[2] + self.indices_distance[2], str(idx), color='r', fontsize=self.indices_size)
            
        self.ax.set_box_aspect([1, 1, 1])
    
        # Apply the current view limits
        self.ax.set_xlim(self.current_xlim)
        self.ax.set_ylim(self.current_ylim)
        self.ax.set_zlim(self.current_zlim)

        self.node1_input.setText(str(self.node1))
        self.node2_input.setText(str(self.node2))
        
        self.canvas.draw()

    def confirm_node(self, node_number):
        try:
            node_index = int(self.node1_input.text() if node_number == 1 else self.node2_input.text())
            if 0 <= node_index < len(self.marks2):
                if node_number == 1:
                    if self.node1 is not None:
                        self.node_colors[self.node1] = 'r'  # Reset previous node1 color to red
                    self.node1 = node_index
                    self.node_colors[self.node1] = 'b'  # Set new node1 color to blue
                    self.counter_Left_and_Right = None  # 重置左右计数
                else:
                    if self.node2 is not None:
                        self.node_colors[self.node2] = 'r'  # Reset previous node2 color to red
                    self.node2 = node_index
                    self.node_colors[self.node2] = 'b'  # Set new node2 color to blue
                
                self.plot_graph()
            else:
                self.status_label.setText("Invalid node index.")
        except ValueError:
            self.status_label.setText("Please enter a valid integer.")

    def add_edge(self):
        try:
            if self.node1 is not None and self.node2 is not None:
                if self.node1 == self.node2:
                    self.status_label.setText("Both nodes are the same node.")
                elif not self.G.has_edge(self.node1, self.node2):
                    self.G.add_edge(self.node1, self.node2)
                    # Add new edge color
                    self.edge_colors[(self.node1, self.node2)] = 'b'
                    self.edge_colors[(self.node2, self.node1)] = 'b'
                    self.status_label.setText(f"Added edge between Node {self.node1} and Node {self.node2}.")
                else:
                    self.status_label.setText("Edge already exists.")
                                
                self.plot_graph()
            else:
                self.status_label.setText("Please confirm both node indices.")

        except ValueError:
            self.status_label.setText("Please enter valid integers for node indices.")

    def delete_edge(self):
        try:
            if self.node1 is not None and self.node2 is not None:
                if self.G.has_edge(self.node1, self.node2):
                    self.G.remove_edge(self.node1, self.node2)
                    # Remove edge color from the edge_colors dictionary
                    self.edge_colors.pop((self.node1, self.node2), None)
                    self.edge_colors.pop((self.node2, self.node1), None)
                    self.status_label.setText(f"Deleted edge between Node {self.node1} and Node {self.node2}.")
                else:
                    self.status_label.setText("Edge does not exist.")

                self.plot_graph()
            else:
                self.status_label.setText("Please confirm both node indices.")

        except ValueError:
            self.status_label.setText("Please enter valid integers for node indices.")

    def center_on_node1(self):
        if self.node1 is not None:
            x, y, z = self.marks2[self.node1]
            self.current_center = [x, y, z]
            self.update_zoom()  # Update zoom based on the new center
        else:
            self.status_label.setText("Please confirm Node 1 first.")

    def move_center_to_next_node(self):
        if self.node1 is not None:
            self.node_colors[self.node1] = 'r' if self.node1 != self.node2 else 'b'  # Set back old node1 color
            self.counter_High_Degree +=1
            if self.counter_High_Degree == len(self.high_degree_list):
                self.counter_High_Degree =0
                self.high_degree_list = self.high_degree_nodes[np.argsort(self.high_degree_nodes)]

            self.node1 = self.high_degree_list[self.counter_High_Degree]
            self.node_colors[self.node1] = 'b'  # Set new node1 color to blue
            x, y, z = self.marks2[self.node1]
            self.current_center = [x, y, z]
            self.update_zoom()
            self.counter_Left_and_Right = None  # 重置左右计数
        else:
            self.status_label.setText("Please confirm Node 1 first.")

    def reset_view(self):
        # Reset to initial view limits but keep the current zoom factor
        self.current_xlim = list(self.initial_xlim)
        self.current_ylim = list(self.initial_ylim)
        self.current_zlim = list(self.initial_zlim)
        self.current_center = [
            (self.current_xlim[0] + self.current_xlim[1]) / 2,
            (self.current_ylim[0] + self.current_ylim[1]) / 2,
            (self.current_zlim[0] + self.current_zlim[1]) / 2
        ]
        self.update_zoom()

    def zoom_in(self):
        self.zoom_factor += self.zoom_step
        self.update_zoom()

    def zoom_out(self):
        self.zoom_factor = max(0.1, self.zoom_factor - self.zoom_step)
        self.update_zoom()

    def reset_zoom(self):
        self.zoom_factor = 1.0
        self.update_zoom()

    def update_zoom(self):
        x_range = (self.initial_xlim[1] - self.initial_xlim[0]) / self.zoom_factor
        y_range = (self.initial_ylim[1] - self.initial_ylim[0]) / self.zoom_factor
        z_range = (self.initial_zlim[1] - self.initial_zlim[0]) / self.zoom_factor

        # Calculate new limits centered on the current center
        self.current_xlim = [self.current_center[0] - x_range / 2, self.current_center[0] + x_range / 2]
        self.current_ylim = [self.current_center[1] - y_range / 2, self.current_center[1] + y_range / 2]
        self.current_zlim = [self.current_center[2] - z_range / 2, self.current_center[2] + z_range / 2]

        self.plot_graph()  

    def save_data(self):
        # 如果 self.file_name 后缀为 '.mark'，将其改为 '.csv'
        if self.file_name.endswith('.mark'):
            self.default_file_name = self.file_name.replace('.mark', '.csv')
        else:
            self.default_file_name = self.file_name
            
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getSaveFileName(self, "Save Data", self.default_file_name, "CSV Files (*.csv);;All Files (*)", options=options)
        
        if file_path:
            # Save all data in a single CSV file
            with open(file_path, 'w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['type', 'id1', 'id2', 'x', 'y', 'z', 'color'])
                # Save nodes
                for i, (x, y, z) in enumerate(self.marks2):
                    writer.writerow(['node', i, '', x, y, z, self.node_colors[i]])
                # Save edges
                for edge in self.G.edges():
                    writer.writerow(['edge', edge[0], edge[1], '', '', '', self.edge_colors.get((edge[0], edge[1]), 'k')])
            
            self.status_label.setText(f"Data saved to {file_path}.")

    def load_data(self):
        options = QFileDialog.Options()
        file_path, _ = QFileDialog.getOpenFileName(self, "Open Data", "", "CSV Files (*.csv);;All Files (*)", options=options)
        if file_path:
            # Load all data from a single CSV file
            self.marks2 = []
            self.node_colors = []
            self.G = nx.Graph()
            self.edge_colors = {}

            # 获取文件名并设置为窗口标题
            self.file_name = os.path.basename(file_path)
            self.setWindowTitle(f"3D Graph - {self.file_name}")
            
            with open(file_path, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Skip header
                for row in reader:
                    data_type = row[0]
                    if data_type == 'node':
                        i, x, y, z, color = int(row[1]), float(row[3]), float(row[4]), float(row[5]), row[6]
                        self.marks2.append([x, y, z])
                        self.node_colors.append(color)
                    elif data_type == 'edge':
                        source, target, color = int(row[1]), int(row[2]), row[6]
                        self.G.add_edge(source, target)
                        self.edge_colors[(source, target)] = color
                        self.edge_colors[(target, source)] = color  # Ensure bidirectional edges have colors

            self.marks2 = np.array(self.marks2)
            self.node1 = 0
            self.node_colors[self.node1] = 'b'
            self.node2 = None
            self.high_degree_list = np.array([node for node in self.G.nodes() if self.G.degree(node) >= 3])
            self.high_degree_list = self.high_degree_list[np.argsort(self.high_degree_list)]
            self.counter_High_Degree = 0  # 节点中心化计数
            self.counter_Left_and_Right = None  # 方向左右键计数
    
            self.azim = -60
            self.elev = 30
            self.ax.view_init(elev=self.elev, azim=self.azim)
            self.zoom_factor = 1.0
            self.initial_xlim = (self.marks2[:, 0].min() - 1, self.marks2[:, 0].max() + 1)
            self.initial_ylim = (self.marks2[:, 1].min() - 1, self.marks2[:, 1].max() + 1)
            self.initial_zlim = (self.marks2[:, 2].min() - 1, self.marks2[:, 2].max() + 1)
            self.reset_view()
            self.status_label.setText(f"Data loaded from {file_path}.")

    def keyPressEvent(self, event):
        if event.key() == Qt.Key_Space:
            self.add_edge()
        elif event.key() == Qt.Key_Backspace:
            self.delete_edge()
        elif event.key() == Qt.Key_Shift:
            self.center_on_node1()
        elif event.key() == Qt.Key_Control:
            self.reset_view()
            
        elif event.key() == Qt.Key_J:
            self.zoom_in()
        elif event.key() == Qt.Key_K:
            self.zoom_out()
        elif event.key() == Qt.Key_L:
            self.reset_zoom()

        elif event.key() == Qt.Key_BracketLeft:
            self.load_data()
        elif event.key() == Qt.Key_BracketRight:
            self.save_data()

        # Adjust azimuth (rotation around z-axis)
        if event.key() == Qt.Key_U:
            self.azim -= 10
            self.ax.view_init(elev=self.elev, azim=self.azim)
            self.plot_graph()
        elif event.key() == Qt.Key_I:
            self.azim += 10
            self.ax.view_init(elev=self.elev, azim=self.azim)
            self.plot_graph()
        # Adjust elevation (rotation around x-axis)
        elif event.key() == Qt.Key_Y:
            self.elev += 10
            self.ax.view_init(elev=self.elev, azim=self.azim)
            self.plot_graph()
        elif event.key() == Qt.Key_H:
            self.elev -= 10
            self.ax.view_init(elev=self.elev, azim=self.azim)
            self.plot_graph()
        
        if self.node1 is not None:
            if event.key() == Qt.Key_Up or event.key() == Qt.Key_W:
                self.node_colors[self.node1] = 'r'
                if self.node2 is not None:
                    self.node_colors[self.node2] = 'b' 
                self.node1 = (self.node1 +1) % len(self.marks2)
                self.counter_Left_and_Right = None
                self.node_colors[self.node1] = 'b'
                self.plot_graph()  # 更新图形
                
            elif event.key() == Qt.Key_Down or event.key() == Qt.Key_S:
                self.node_colors[self.node1] = 'r' 
                if self.node2 is not None:
                    self.node_colors[self.node2] = 'b' 
                self.node1 = (self.node1 -1) % len(self.marks2)
                self.counter_Left_and_Right = None
                self.node_colors[self.node1] = 'b'
                self.plot_graph()  # 更新图形
            
            elif event.key() == Qt.Key_Left or event.key() == Qt.Key_A:
                if self.counter_Left_and_Right == None:
                    self.counter_Left_and_Right = 0  
                self.counter_Left_and_Right -=1
                if self.node2 is not None:
                    self.node_colors[self.node2] = 'r' 
                self.node_colors[self.node1] = 'b'
                nearest_indices = np.argsort(np.linalg.norm(self.marks2 - self.marks2[self.node1], axis=1))
                nearest_indices = nearest_indices[nearest_indices != self.node1][:5]
                # nearest_indices = nearest_indices[np.argsort(nearest_indices)]
                self.node2 = nearest_indices[self.counter_Left_and_Right %5]
                self.node_colors[self.node2] = 'b'
                self.plot_graph()  # 更新图形
            
            elif event.key() == Qt.Key_Right or event.key() == Qt.Key_D:
                if self.counter_Left_and_Right == None:
                    self.counter_Left_and_Right = -1   
                self.counter_Left_and_Right +=1
                if self.node2 is not None:
                    self.node_colors[self.node2] = 'r' 
                self.node_colors[self.node1] = 'b'
                nearest_indices = np.argsort(np.linalg.norm(self.marks2 - self.marks2[self.node1], axis=1))
                nearest_indices = nearest_indices[nearest_indices != self.node1][:5]
                # nearest_indices = nearest_indices[np.argsort(nearest_indices)]
                self.node2 = nearest_indices[self.counter_Left_and_Right %5]
                self.node_colors[self.node2] = 'b'
                self.plot_graph()  # 更新图形

            elif event.key() == Qt.Key_R or event.key() == Qt.Key_PageUp:
                if self.node2 is not None:
                    temp = self.node1
                    self.node1 = self.node2
                    self.node2 = temp
                    self.counter_Left_and_Right = None
                    self.plot_graph()  # 更新图形
                    
            elif event.key() == Qt.Key_F or event.key() == Qt.Key_PageDown:
                self.move_center_to_next_node()



In [5]:
'''
# 定义批处理
import os
import csv
import numpy as np
from PyQt5.QtWidgets import QFileDialog

def save_G2(G2, marks2, save_file_path, node_colors='r', edge_colors='k'):
    """
    保存 G2 拓扑图的数据到指定文件。
    """
    with open(save_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['type', 'id1', 'id2', 'x', 'y', 'z', 'color'])
        # 保存节点
        for i, (x, y, z) in enumerate(marks2):
            writer.writerow(['node', i, '', x, y, z, node_colors])
        # 保存边
        for edge in G2.edges():
            writer.writerow(['edge', edge[0], edge[1], '', '', '', edge_colors])
    print(f"Data saved to {save_file_path}.")

def process_and_save(obj_file_path, mark_file_path1, mark_file_path2, save_file_path):
    # 检测编码
    obj_encoding = detect_encoding(obj_file_path)
    mark_encoding2 = detect_encoding(mark_file_path2)

    # 读取点云并插值加密，读取两个marks文件，点云和marks相互配准
    obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
    checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces)
    centered_vertices = center_vertices(checked_vertices)
    marks2 = load_mark_file(mark_file_path2, mark_encoding2)

    # 姿态调整，生成marks2的拓扑图
    aligned_points, final_rotation_matrix = align_point_cloud(np.asarray(centered_vertices))
    aligned_marks2 = np.asarray(marks2).dot(final_rotation_matrix)
    G2 = generate_topology(aligned_marks2)

    # 保存结果
    save_G2(G2, aligned_marks2, save_file_path)

def batch_process_files(obj_file_dir, mark_file_dir1, mark_file_dir2, save_file_dir):
    """
    批量处理数据文件，将结果保存到指定位置。

    参数:
    obj_file_dir: 存放 .obj 文件的目录路径。
    mark_file_dir1: 存放 .mark 文件的第一个目录路径。
    mark_file_dir2: 存放 .mark 文件的第二个目录路径。
    save_file_dir: 保存处理结果的目录路径。
    """
    # 确保保存结果的目录存在
    os.makedirs(save_file_dir, exist_ok=True)

    count1 = 1
    count2 = 0
    
    # 遍历 obj_file_dir 目录中的所有 .obj 文件
    for obj_filename in os.listdir(obj_file_dir):
        if obj_filename.endswith('.obj'):
            print(f"正在处理第{count1}个预备体")
            
            # 构造 obj 文件路径
            obj_file_path = os.path.join(obj_file_dir, obj_filename)
            
            # 构造对应的 mark 文件名
            mark_file_name1 = f"{obj_filename}.mark"
            mark_file_name2 = f"{obj_filename}.mark"
            
            # 构造 mark 文件路径
            mark_file_path1 = os.path.join(mark_file_dir1, mark_file_name1)
            mark_file_path2 = os.path.join(mark_file_dir2, mark_file_name2)
            
            # 构造保存文件路径
            save_file_name = f"{os.path.splitext(obj_filename)[0]}.csv"  # 更改为 .csv 后缀
            save_file_path = os.path.join(save_file_dir, save_file_name)
            
            # 检查 mark 文件是否存在
            if os.path.exists(mark_file_path1) and os.path.exists(mark_file_path2):
                # 调用处理函数
                process_and_save(obj_file_path, mark_file_path1, mark_file_path2, save_file_path)
                count2 += 1
            else:
                print(f"Warning: Missing mark file(s) for {obj_filename}")

            print(f"已成功处理{count2}个预备体")
            count1 += 1



# 执行批处理
# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/肩台外侧点-0715/"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())

# 设置输入文件路径
obj_file_dir = "Aug22"
mark_file_dir1 = "Aug22"
mark_file_dir2 = "Aug22/问题数据" 
save_file_dir = "Aug22/问题数据"

# 整
batch_process_files(obj_file_dir, mark_file_dir1, mark_file_dir2, save_file_dir)
'''

'\n# 定义批处理\nimport os\nimport csv\nimport numpy as np\nfrom PyQt5.QtWidgets import QFileDialog\n\ndef save_G2(G2, marks2, save_file_path, node_colors=\'r\', edge_colors=\'k\'):\n    """\n    保存 G2 拓扑图的数据到指定文件。\n    """\n    with open(save_file_path, \'w\', newline=\'\') as file:\n        writer = csv.writer(file)\n        writer.writerow([\'type\', \'id1\', \'id2\', \'x\', \'y\', \'z\', \'color\'])\n        # 保存节点\n        for i, (x, y, z) in enumerate(marks2):\n            writer.writerow([\'node\', i, \'\', x, y, z, node_colors])\n        # 保存边\n        for edge in G2.edges():\n            writer.writerow([\'edge\', edge[0], edge[1], \'\', \'\', \'\', edge_colors])\n    print(f"Data saved to {save_file_path}.")\n\ndef process_and_save(obj_file_path, mark_file_path1, mark_file_path2, save_file_path):\n    # 检测编码\n    obj_encoding = detect_encoding(obj_file_path)\n    mark_encoding2 = detect_encoding(mark_file_path2)\n\n    # 读取点云并插值加密，读取两个marks文件，点云和marks相互配准\n    obj_vertices, obj_fa

In [6]:
'''
mark_file_path2 = "Aug22/问题数据/103.obj.mark" 
obj_encoding = detect_encoding(obj_file_path)
mark_encoding2 = detect_encoding(mark_file_path2)
obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces)
centered_vertices = center_vertices(checked_vertices)
marks2 = load_mark_file(mark_file_path2, mark_encoding2)
aligned_points, final_rotation_matrix = align_point_cloud(np.asarray(centered_vertices))
aligned_marks2 = np.asarray(marks2).dot(final_rotation_matrix)

print(np.asarray(marks2))
'''

'\nmark_file_path2 = "Aug22/问题数据/103.obj.mark" \nobj_encoding = detect_encoding(obj_file_path)\nmark_encoding2 = detect_encoding(mark_file_path2)\nobj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)\nchecked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces)\ncentered_vertices = center_vertices(checked_vertices)\nmarks2 = load_mark_file(mark_file_path2, mark_encoding2)\naligned_points, final_rotation_matrix = align_point_cloud(np.asarray(centered_vertices))\naligned_marks2 = np.asarray(marks2).dot(final_rotation_matrix)\n\nprint(np.asarray(marks2))\n'

In [8]:

# 单独处理一个预备体
# 设置各种地址
import os

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/肩台外侧点-0715/"
os.chdir(new_dir)

# 设置输入文件路径
obj_file_path = 'Aug15/肩台外侧标志点_0815/0812-1-5_4.obj'
mark_file_path1 = 'Aug15/肩台外侧标志点_0815/0812-1-5_4.obj.mark'
mark_file_path2 = 'Aug12/多分类mark2/4.obj.mark'

obj_encoding = detect_encoding(obj_file_path)
mark_encoding1 = detect_encoding(mark_file_path1)
mark_encoding2 = detect_encoding(mark_file_path2)

obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces)
centered_vertices = center_vertices(checked_vertices)
marks1 = load_mark_file(mark_file_path1, mark_encoding1)
marks2 = load_mark_file(mark_file_path2, mark_encoding2)
marks = marks1 + marks2

aligned_points, final_rotation_matrix = align_point_cloud(np.asarray(centered_vertices))
aligned_marks2 = np.asarray(marks2).dot(final_rotation_matrix)

# 启动GUI
if __name__ == '__main__':
    app = QApplication(sys.argv)
    
    # 测试数据，正式使用时删掉这一行
    # aligned_marks2 = np.random.rand(10, 3) * 10
    
    # 真实数据，正式使用时要解除注释
    aligned_marks2 = np.asarray(marks2).dot(final_rotation_matrix)

    marks2_file_name = os.path.basename(mark_file_path2)
    
    editor = TopologyEditor(aligned_marks2, marks2_file_name)

    # 设置窗口标题为文件名
    editor.setWindowTitle(f"3D Graph - {marks2_file_name}")
    
    editor.show()
    
    sys.exit(app.exec_())


使用了 1 轮插值来让点云点数满足要求


SystemExit: 0